In [4]:
import numpy as np

# Load matrices from TSV files
matrix1 = np.loadtxt("output_factors.lhs.tsv", delimiter="\t")
matrix2 = np.loadtxt("output_factors.rhs.tsv", delimiter="\t")
matrix3 = np.loadtxt("matrix.tsv", delimiter="\t")

# Perform matrix multiplication
result = np.matmul(matrix1, matrix2)

# Perform subtraction
final_result = result - matrix3
print(matrix3.shape)
# Save final result to a new file
# np.savetxt("final_result.tsv", final_result, delimiter="\t", fmt="%d")
count = np.count_nonzero(final_result == -1)
print(count)

(100, 150)
6540


# SGD simple works 

In [252]:
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T


# Reads the movie lens data and creates a matrix out of it 

import numpy as np
import pandas as pd
ratings = pd.read_csv("/home/cris-musa/Documents/Dataset/ratings.csv")
movies = pd.read_csv("/home/cris-musa/Documents/Dataset/movies.csv")
# print(ratings)
# print(movies)

R = np.zeros(shape=(610,193609),dtype=float)
for i in range(100835):
    user = ratings["userId"][i]-1
    movie = ratings["movieId"][i]-1
    R[user][movie]=ratings["rating"][i]
print(R)


print("")


K = 3




R = numpy.array(R)
# N: num of User
N = len(R)
# M: num of Movie
M = len(R[0])

 
P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K)

nR = numpy.dot(nP, nQ.T)
print(nR)

# print(R-nR)

[[4.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 2.  2.  ... 0.  0.  0. ]
 [3.  0.  0.  ... 0.  0.  0. ]
 [5.  0.  0.  ... 0.  0.  0. ]]



KeyboardInterrupt: 

In [ ]:
print(np.linalg.norm(R-nR)**2)

In [239]:
import numpy as np

def matrix_factorization_PALM(R, k, max_iter=1000, tol=1e-4):
    """
    Matrix factorization using Proximal Alternating Linear Minimization (PALM)
    
    Parameters:
        R (np.ndarray): Input matrix to factorize
        k (int): Number of latent factors
        max_iter (int): Maximum number of iterations
        tol (float): Tolerance for convergence
        lambda_ (float): Regularization term
        
    Returns:
        (np.ndarray, np.ndarray): The two factor matrices
    """
    m, n = R.shape
    X = np.random.rand(m, k)
    Y = np.random.rand(k, n)
    
    # Set the step size
    eta = 1 / (np.linalg.norm(X, 2) * np.linalg.norm(Y, 2))
#     print(eta)
    
    for i in range(max_iter):
#         print(i)
    #         X_prev = X.copy()
    #         Y_prev = Y.copy()

        # Minimize with respect to X

        for j in range(m):
            YYt = Y @ Y.T
            RYt = R @ Y.T
            L = max(np.linalg.norm(YYt), 1e-4)
            eta = 1/(1.1*L)
            grad_X = X @ YYt - RYt
            X = X-grad_X * eta


        # Minimize with respect to Y
        for j in range(n):
            XtX = X.T @ X
            RtX = R.T @ X
            L = max(np.linalg.norm(XtX), 1e-4)
            eta = 1/(1.1*L)
            grad_Yt = Y.T @ XtX - RtX
            Yt = Y.T-grad_Yt *eta


        
        ell = np.linalg.norm(R - X @ Y)**2
        ell0 = ell
        # Check for convergence
    #         if (abs(ell - ell0) < tol):
    #             break   
    return X, Y


In [192]:
R = [

    [5,3,0,1],

     [4,0,0,1],

     [1,1,0,5],

     [1,0,0,4],
    
     [1,2,0,5],

    ]

R = numpy.array(R)
# N: num of User
# N = len(R)
# # M: num of Movie
# M = len(R[0])
# Num of Features
K = 3
nP, nQ = matrix_factorization_PALM(R, K)

nR = numpy.dot(nP, nQ)
print(nR)

[[ 4.65466402  3.44029303  0.33928606  0.02295611]
 [ 3.41104293  0.75090264  0.57863917 -0.66631034]
 [-0.19626591  2.52520322  1.17530859  1.6154575 ]
 [-0.06222382  1.35430357  1.04361478  0.99469687]
 [-0.08631211  3.38501542  1.06728107  1.92654494]]


In [149]:
import numpy as np
matrix = np.random.randint(6, size=(100, 150), dtype=np.int8)
matrix

array([[0, 5, 0, ..., 0, 1, 4],
       [2, 1, 5, ..., 4, 0, 5],
       [2, 0, 2, ..., 5, 1, 0],
       ...,
       [1, 5, 2, ..., 2, 0, 1],
       [2, 4, 2, ..., 4, 2, 3],
       [2, 3, 2, ..., 3, 0, 2]], dtype=int8)

# Working Great

In [220]:
import numpy as np

def matrix_factorization_PALM(R, k, max_iter=1000, tol=1e-4):
    """
    Matrix factorization using Proximal Alternating Linear Minimization (PALM)
    
    Parameters:
        R (np.ndarray): Input matrix to factorize
        k (int): Number of latent factors
        max_iter (int): Maximum number of iterations
        tol (float): Tolerance for convergence
        lambda_ (float): Regularization term
        
    Returns:
        (np.ndarray, np.ndarray): The two factor matrices
    """
    m, n = R.shape
    X = np.random.rand(m, k) + 0.1 * np.random.randn(m, k)
    Y = np.random.rand(k, n) + 0.1 * np.random.randn(k, n)
    
    # Set the step size
#     eta = 1 / (np.linalg.norm(X, 2) * np.linalg.norm(Y, 2))
#     print(eta)
    
    for i in range(max_iter):
        
        #Updating X
        YYt = Y @ Y.T
        RYt = R @ Y.T
        L = max(np.linalg.norm(YYt), 1e-4)
        eta = 1/(1.1*L)
        grad_X = X @ YYt - RYt
        X = X-grad_X * eta
        
        
        #updating Y
        XtX = X.T @ X
        RtX = R.T @ X
        L = max(np.linalg.norm(XtX), 1e-4)
        eta = 1/(1.1*L)
        grad_Yt = Y.T @ XtX - RtX
        Yt = Y.T-grad_Yt *eta
        
        
        
        ell = np.linalg.norm(R - X @ Y)**2
        ell0 = ell
        # Check for convergence
#         if (abs(ell - ell0) < tol):
#             break    

    return X, Y


In [222]:
R = np.random.randint(6, size=(5, 4), dtype=np.int8)
# N: num of User
# N = len(R)
# # M: num of Movie
# M = len(R[0])
# Num of Features
print(R)
print("")
K = 3
nP, nQ = matrix_factorization_PALM(R, K)

nR = numpy.dot(nP, nQ)
print(nR)

[[1 2 2 1]
 [1 3 1 5]
 [0 1 4 4]
 [3 1 2 2]
 [2 4 0 4]]

[[1.04014905 2.07265776 1.65702102 1.2911812 ]
 [0.7370133  2.5240728  3.24660168 3.09268713]
 [0.02580682 1.04670029 3.77955294 4.18715345]
 [2.98282984 0.96892896 2.14667048 1.87548062]
 [1.70543883 3.46693408 2.51632326 1.86370041]]


array([[4, 2, 0, ..., 0, 3, 2],
       [5, 4, 5, ..., 1, 3, 0],
       [5, 3, 3, ..., 3, 1, 4],
       ...,
       [5, 3, 1, ..., 4, 0, 1],
       [1, 0, 4, ..., 2, 3, 4],
       [2, 5, 3, ..., 3, 1, 3]], dtype=int8)

# Gen

In [144]:
import numpy as np

def matrix_factorization_PALM(R, k, max_iter=100000, tol=1e-4, lambda_=0.1):
    """
    Matrix factorization using Proximal Alternating Linear Minimization (PALM) with regularization
    
    Parameters:
        R (np.ndarray): Input matrix to factorize
        k (int): Number of latent factors
        max_iter (int): Maximum number of iterations
        tol (float): Tolerance for convergence
        lambda_ (float): Regularization term
        
    Returns:
        (np.ndarray, np.ndarray): The two factor matrices
    """
    m, n = R.shape
    X = np.random.rand(m, k)
    Y = np.random.rand(k, n)
    
    # Set the step size
    eta = 1 / (np.linalg.norm(X, 2) * np.linalg.norm(Y, 2))
    #print(eta)
    
    for i in range(max_iter):
        
        #Updating X
        YYt = Y @ Y.T
        RYt = R @ Y.T
        L = max(np.linalg.norm(YYt), 1e-4)
        eta = 1/(1.1*L)
        grad_X = X @ YYt - RYt + lambda_*X
        X = X-grad_X * eta
        
        #updating Y
        XtX = X.T @ X
        RtX = R.T @ X
        L = max(np.linalg.norm(XtX), 1e-4)
        eta = 1/(1.1*L)
        grad_Yt = Y.T @ XtX - RtX + lambda_*Y
        Yt = Y.T-grad_Yt *eta
        Y=Yt.T
        
        ell = np.linalg.norm(R - X @ Y)**2 + lambda_*(np.linalg.norm(X)**2 + np.linalg.norm(Y)**2)
        ell0 = ell
        # Check for convergence
#         if (abs(ell - ell0) < tol):
#             break    

    return X, Y


In [145]:
R = [

    [5,3,0,1],

     [4,0,0,1],

     [1,1,0,5],

     [1,0,0,4],

    ]

R = numpy.array(R)
# N: num of User
# N = len(R)
# # M: num of Movie
# M = len(R[0])
# Num of Features
print(matrix)
K = 3
nP, nQ = matrix_factorization_PALM(matrix, K)

nR = numpy.dot(nP, nQ)
print("")
print(nR)

ValueError: operands could not be broadcast together with shapes (4,3) (3,4) 

In [ ]:
import numpy as np

def matrix_factorization_PALM(R, k, max_iter=100000, tol=1e-4, lambda_=0.1, alpha = 0.5, beta = 0.8, gamma = 0.9):
    """
    Matrix factorization using Proximal Alternating Linear Minimization (PALM)
    
    Parameters:
        R (np.ndarray): Input matrix to factorize
        k (int): Number of latent factors
        max_iter (int): Maximum number of iterations
        tol (float): Tolerance for convergence
        lambda_ (float): Regularization term
        
    Returns:
        (np.ndarray, np.ndarray): The two factor matrices
    """
    m, n = R.shape
    X = np.random.rand(m, k) + 0.1 * np.random.randn(m, k)
    Y = np.random.rand(k, n) + 0.1 * np.random.randn(k, n)

    # Set the step size
    eta = 1 / (np.linalg.norm(X, 2) * np.linalg.norm(Y, 2))
    v_x = 0
    v_y = 0
    for i in range(max_iter):
        X_old = X
        Y_old = Y
        YYt = Y @ Y.T
        RYt = R @ Y.T
        L = max(np.linalg.norm(YYt), 1e-4)
        eta = 1/(1.1*L)
        grad_X = (X @ YYt - RYt) + lambda_ * np.sign(X)
        v_x = gamma*v_x + eta*grad_X
        X = X-v_x
        while (np.linalg.norm(R-X@Y)**2) > (np.linalg.norm(R-X_old@Y_old)**2- alpha*eta*(np.linalg.norm(grad_X)**2)):
            eta = eta*beta
            X = X_old - eta*grad_X
            
        X = np.maximum(X, lambda_)
        XtX = X.T @ X
        RtX = R.T @ X
        L = max(np.linalg.norm(XtX), 1e-4)
        eta = 1/(1.1*L)
        grad_Yt = (Y.T @ XtX - RtX) + lambda_ * np.sign(Y)
        v_y = gamma*v_y + eta*grad_Yt
        Yt = Y.T-v_y
        Y = Yt.T
        while (np.linalg.norm(R-X@Y)**2) > (np.linalg.norm(R-X_old@Y_old)**2- alpha*eta*(np.l.linalg.norm(grad_Yt)**2)):
            eta = eta*


In [203]:
matrix = np.random.randint(6, size=(100, 150), dtype=np.int8)
matrix_factorization_PALM(matrix,10)

UnboundLocalError: local variable 'Yt' referenced before assignment

In [256]:
import numpy as np

# Step 1: Create a matrix of user-item ratings
ratings = np.array([[5, 3, 0, 1], [4, 0, 0, 1], [1, 1, 0, 5], [1, 0, 0, 4], [0, 1, 5, 4]])

# Step 2: Initialize matrices for user and item latent representations
num_users, num_items = ratings.shape
latent_dim = 3
user_latent = np.random.random((num_users, latent_dim))
item_latent = np.random.random((num_items, latent_dim))

# Step 3 and 4: Iteratively update the user and item latent representations
for i in range(100):
    # Fix item latent representations and solve for user latent representations
    for user in range(num_users):
        user_latent[user] = np.linalg.lstsq(item_latent, ratings[user])[0]

    # Fix user latent representations and solve for item latent representations
    for item in range(num_items):
        item_latent[item] = np.linalg.lstsq(user_latent, ratings[:, item])[0]

# Step 5: Use the learned latent representations to make recommendations
user_predictions = np.dot(user_latent, item_latent.T)
print(user_predictions)


[[ 5.32652372  2.18816428  0.18504005  1.00294508]
 [ 3.46567149  1.32850063 -0.30280243  0.99518063]
 [ 1.19208569  0.52241748  0.10885441  5.00173251]
 [ 0.86190988  0.34333338 -0.07825527  3.9987545 ]
 [-0.05108206  1.1270053   4.97105194  3.99953927]]


/tmp/ipykernel_357771/622531850.py:16: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  user_latent[user] = np.linalg.lstsq(item_latent, ratings[user])[0]
/tmp/ipykernel_357771/622531850.py:20: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  item_latent[item] = np.linalg.lstsq(user_latent, ratings[:, item])[0]
